# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [2]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [3]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [4]:
a = "This is a" +\
    "string"

In [5]:
a = ("This is a " +
    "string")

In [6]:
a = """
This is one line
This is another
"""

In [7]:
a

'\nThis is one line\nThis is another\n'

In [8]:
pd.io.sql.read_sql(
''' --- This is how you make a comment in SQL
SELECT p.id, p.size As size_foo_bar --- renaming a column
    FROM properties As p --- Put all SQL statement in uppercase and everything else not. Rename table to p.
    LIMIT 10
;
''', con = db) # Specifying connection to db above

,id,size_foo_bar
0,2121978635,557.0
1,89239580,1050.0
2,15131782,937.0
3,15179502,1574.0
4,52266124,1205.0
5,2100994004,400.0
6,15067755,541.0
7,15112556,1175.0
8,15133321,3094.0
9,61288341,1405.0


### `SELECT`

In [9]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [19]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [22]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000 / 1000 AS sale_price --- / 1000 / 1000 is the same as / 1000000
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [31]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [32]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [34]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [35]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [43]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count --- COUNT() is a function
    FROM properties
    GROUP BY beds --- Removing this line would just return a total count only.
;
''', con = db)

,count
0,164
1,142
2,316
3,226
4,103
5,28
6,9
7,6
8,3
9,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [46]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price --- Starter code file was missing "baths"
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,baths,mean_sale_price
0,NaN,1.739026
1,1.00,0.987656
2,1.10,1.420000
3,1.25,1.600000
4,1.50,1.223378
...,...,...
16,6.50,16.000000
17,7.00,0.999000
18,7.50,5.530000
19,8.00,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [68]:
# TODO
pd.io.sql.read_sql(
'''
WITH t AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

SELECT 
    AVG(t.sale_price)
    FROM properties AS p
        JOIN t on p.id = t.id
    WHERE p.is_a_studio = 1
;
''', con = db)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: Execution failed on sql '
WITH t AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

SELECT 
    AVG(t.sale_price)
    FROM properties AS p
        JOIN t on p.id = t.id
    WHERE p.is_a_studio = 1
;
': near "SELECT": syntax error

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [63]:
# TODO
pd.io.sql.read_sql(
'''
SELECT a.id AS id_a, b. id AS id_b, a.address AS addy_a, b.address AS addy_b
    FROM properties AS a
    JOIN properties AS b ON b.latitude = a.latitude AND b.longitude = a.longitude
    WHERE a.address <> b.address
;
''', con = db)

,id_a,id_b,addy_a,addy_b
0,52266124,53153575,"88 King St APT 317, San Francisco, CA","88 King St APT 419, San Francisco, CA"
1,119685526,119685188,"1902 Filbert St, San Francisco, CA","1902A Filbert St, San Francisco, CA"
2,15076711,15076699,"1 Daniel Burnham Ct APT 1404, San Francisco, CA","1 Daniel Burnham Ct APT 1202, San Francisco, CA"
3,15068514,15068551,"1177 California St APT 702, San Francisco, CA","1177 California St APT 501, San Francisco, CA"
4,15068514,15068580,"1177 California St APT 702, San Francisco, CA","1177 California St APT 423, San Francisco, CA"
...,...,...,...,...
209,60825591,61288454,"77 Dow Pl APT 500, San Francisco, CA","77 Dow Pl APT 507, San Francisco, CA"
210,67395015,63107226,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
211,67395015,67394898,"8400 Oceanview Ter APT 218, San Francisco, CA","8300 Oceanview Ter APT 306, San Francisco, CA"
212,63197318,63197354,"765 Market St APT 27B, San Francisco, CA","765 Market St APT 31H, San Francisco, CA"
